In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely import wkt

import pygris
from pygris.data import get_census
from pygris import tracts
from pygris.utils import erase_water

In [2]:
# uploading precinct demographic estimates

precinct_demographics = pd.read_csv('../data/input/precinct-demo-data.csv')

In [3]:
# getting list of cure precincts

cure_data = pd.read_csv('../data/output/programMerge.csv')

cure_precincts_all = list(cure_data['precinct'].unique())
cure_precincts_pre_covid = list(cure_data[cure_data['yearCure'] <= 2020]['precinct'].unique())

In [4]:
# subsetting to list of relevant demographics

demo_list = ['% Male', '% Female', '% Hispanic or Latino', '% Asian alone', 
             '% American Indian or Alaska Native alone', '% Black or African American alone', 
             '% Native Hawaiian or other Pacific Islander alone', '% White alone', 
             '% Some other race alone', '% Two or more races', '% Adults with no high school diploma', 
             '% Adults with high school diploma (includes equivalency)', 
             '% Adults with Bachelor\'s degree or higher','% Unemployed (civilians 16 and older)',
             '% In the civilian labor force (16 and older)', '% With private health insurance (civilian, noninstitutionalized)',
             '% With public coverage (civilian, noninstitutionalized)',
             '% With no health insurance coverage (civilian, noninstitutionalized)'
            ]

precinct_demographics = precinct_demographics[['precinct'] + ['Total population'] + demo_list]

In [5]:
# creating unemployment rate

precinct_demographics['% Unemployed (civilians 16 and older)'] = precinct_demographics['Total population'] * precinct_demographics['% Unemployed (civilians 16 and older)']
precinct_demographics['% In the civilian labor force (16 and older)'] = precinct_demographics['Total population'] * precinct_demographics['% In the civilian labor force (16 and older)']
precinct_demographics['% Unemployed (civilians 16 and older)'] = round(((precinct_demographics['% Unemployed (civilians 16 and older)'] / precinct_demographics['% In the civilian labor force (16 and older)'])*100),2)   

In [6]:
# creating median income by precinct

my_census_api_key = '279a5c5d33804d3a2856d4a14845778b06963a9b'

# uploading nyc census tracts

nyc_counties = ["New York", "Bronx", "Richmond", 
                "Kings", "Queens"]

nyc_tracts = tracts(state = "NY", county = nyc_counties, cb = True, year = 2021, cache = True) 
nyc_tracts = erase_water(nyc_tracts, area_threshold = 0.9) # erasing water
nyc_tracts['2020 Tract ID'] = nyc_tracts['TRACTCE'].astype(int).astype(str) + '-' + nyc_tracts['COUNTYFP'].astype(int).astype(str) # unique tract identifier

# uploading precinct geometries
precinct_geographies = pd.read_csv('../data/input/precinct-demo-data.csv')
precinct_geographies['geometry'] = precinct_geographies['geometry'].apply(wkt.loads) # converting to GeoDataFrame
precinct_geographies = GeoDataFrame(precinct_geographies, crs="EPSG:4326", geometry='geometry')

# retrieving median income by census tract
med_income_by_tract = get_census(dataset = "acs/acs5/profile", # acs 5yr link
                                 variables = ['DP03_0062E'], # retrieves median income in 2021 inflation-adjusted dollars
                                 year = 2021, # 2017-2021 acs
                                 params = {
                                   "for": "tract:*", # tract-level
                                   "in": "state:36", # NY
                                   "key": my_census_api_key}, 
                                 guess_dtypes = True,
                                 return_geoid = True)

med_income_by_tract = nyc_tracts.merge(med_income_by_tract, how = 'inner', on = 'GEOID') # merging with nyc_tracts to get info just for NYC 
med_income_by_tract = med_income_by_tract[['DP03_0062E','GEOID']].rename(columns={'DP03_0062E': 'Median household income'}) 

# adding unique identifier column: '2020 Tract ID'
med_income_by_tract['Census Tract'] = med_income_by_tract['GEOID'].str[5:].astype(int)
med_income_by_tract['County FIPS'] = med_income_by_tract['GEOID'].str[2:5].astype(int)
med_income_by_tract['2020 Tract ID'] = med_income_by_tract['Census Tract'].astype(str) + '-' + med_income_by_tract['County FIPS'].astype(str)

# performing overlay funcion to determine share of precinct that each tract takes up
nyc_tracts['area_tract'] = nyc_tracts.area
precinct_geographies['area_precinct'] = precinct_geographies.area
geo_joined = gpd.overlay(precinct_geographies, nyc_tracts, how='union')
geo_joined['area_joined'] = geo_joined.area
geo_joined['tract_area_as_a_share_of_precinct_area'] = round((geo_joined['area_joined'] / geo_joined['area_precinct']),4)
geo_joined = geo_joined[['precinct','2020 Tract ID','geometry','area_precinct','area_tract','area_joined','tract_area_as_a_share_of_precinct_area']]

# adding median income by tract
med_income_dict = pd.Series(list(med_income_by_tract['Median household income'].values),index=list(med_income_by_tract['2020 Tract ID'])).to_dict()
geo_joined['tract_med_income'] = geo_joined['2020 Tract ID'].astype(str).str.findall('|'.join([fr'\b{w}\b' for w in med_income_dict.keys()])).apply(", ".join).map(med_income_dict)

# weighted average 
geo_joined['weighted_med_income'] = round(geo_joined['tract_area_as_a_share_of_precinct_area']*geo_joined['tract_med_income'])

# adding median by precinct to precinct_demographics df
precinct_demographics.index = precinct_demographics['precinct']
med_income_by_precinct = geo_joined.groupby('precinct').sum()['weighted_med_income'] # finding 'weighted average' median income by precinct
precinct_demographics['Median household income'] = med_income_by_precinct

Using FIPS code '36' for input 'NY'
Using FIPS code '061' for input 'New York'
Using FIPS code '005' for input 'Bronx'
Using FIPS code '085' for input 'Richmond'
Using FIPS code '047' for input 'Kings'
Using FIPS code '081' for input 'Queens'


/var/folders/0x/wbh6lcrn3t7046vw2zxtbrhxrc0v0g/T/ipykernel_23316/2929666216.py:39: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nyc_tracts['area_tract'] = nyc_tracts.area
/var/folders/0x/wbh6lcrn3t7046vw2zxtbrhxrc0v0g/T/ipykernel_23316/2929666216.py:40: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  precinct_geographies['area_precinct'] = precinct_geographies.area
/var/folders/0x/wbh6lcrn3t7046vw2zxtbrhxrc0v0g/T/ipykernel_23316/2929666216.py:41: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4269

  geo_joined = gpd.overlay(precinct_geographies, nyc

In [8]:
# creating column indicating whether a precinct is in Cure or not 

precinct_demographics['Cure Precinct'] = ''

for ind in precinct_demographics.index:
    
    if precinct_demographics['precinct'][ind] in (cure_precincts_all):
        
        precinct_demographics['Cure Precinct'][ind] = 'Yes'
        
    else:
        
        precinct_demographics['Cure Precinct'][ind] = 'No'
        
# creating column indicating whether a precinct is in Cure or not (pre-Covid)

precinct_demographics['Cure Precinct Pre-Covid'] = ''

for ind in precinct_demographics.index:
    
    if precinct_demographics['precinct'][ind] in (cure_precincts_pre_covid):
        
        precinct_demographics['Cure Precinct Pre-Covid'][ind] = 'Yes'
        
    else:
        
        precinct_demographics['Cure Precinct Pre-Covid'][ind] = 'No'

/var/folders/0x/wbh6lcrn3t7046vw2zxtbrhxrc0v0g/T/ipykernel_23316/1333214071.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precinct_demographics['Cure Precinct'][ind] = 'No'
/var/folders/0x/wbh6lcrn3t7046vw2zxtbrhxrc0v0g/T/ipykernel_23316/1333214071.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precinct_demographics['Cure Precinct'][ind] = 'Yes'
/var/folders/0x/wbh6lcrn3t7046vw2zxtbrhxrc0v0g/T/ipykernel_23316/1333214071.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [9]:
# performing t-test for each demographic (Cure vs No Cure)

t_stat_dict = {}
p_val_dict = {}

group1 = precinct_demographics[precinct_demographics['Cure Precinct'] == 'Yes']
group2 = precinct_demographics[precinct_demographics['Cure Precinct'] == 'No']

for demo in ['Total population'] + demo_list + ['Median household income']:
    
    cure_distr = group1[demo].dropna()
    no_cure_distr = group2[demo].dropna()
    
    t_stat, p_val = stats.ttest_ind(cure_distr, no_cure_distr)
    
    t_stat_dict[demo] = round(t_stat,2)
    p_val_dict[demo] = round(p_val,2)

In [10]:
# performing t-test for each demographic (Cure Pre-Covid vs not Pre-Covid)

t_stat_dict_pre_covid = {}
p_val_dict_pre_covid = {}

group1 = precinct_demographics[precinct_demographics['Cure Precinct Pre-Covid'] == 'Yes']
group2 = precinct_demographics[precinct_demographics['Cure Precinct Pre-Covid'] == 'No']

for demo in ['Total population'] + demo_list + ['Median household income']:
    
    cure_distr = group1[demo].dropna()
    no_cure_distr = group2[demo].dropna()
    
    t_stat, p_val = stats.ttest_ind(cure_distr, no_cure_distr)
    
    t_stat_dict_pre_covid[demo] = round(t_stat,2)
    p_val_dict_pre_covid[demo] = round(p_val,2)

In [11]:
# grouping by Cure/ non-Cure precincts and Cure Pre-Covid/ non Pre-Covid

cure_precinct_demographics = precinct_demographics.groupby('Cure Precinct').mean().drop(columns=['precinct']).round(1)
cure_precinct_demographics_pre_covid = precinct_demographics.groupby('Cure Precinct Pre-Covid').mean().drop(columns=['precinct']).round(1)


In [12]:
# cleaning table columns and values

for demo in demo_list: # adding % to end of values
            
    cure_precinct_demographics[demo] = cure_precinct_demographics[demo].apply(lambda x: f"{x}%")
    cure_precinct_demographics_pre_covid[demo] = cure_precinct_demographics_pre_covid[demo].apply(lambda x: f"{x}%")

# rounding and adding $ to median income
cure_precinct_demographics['Median household income'] = round(cure_precinct_demographics['Median household income']) 
cure_precinct_demographics_pre_covid['Median household income'] = round(cure_precinct_demographics_pre_covid['Median household income']) 
cure_precinct_demographics['Median household income'] = cure_precinct_demographics['Median household income'].apply(lambda x: f"${x}")
cure_precinct_demographics_pre_covid['Median household income'] = cure_precinct_demographics_pre_covid['Median household income'].apply(lambda x: f"${x}")

# dropping % from demo names
cure_precinct_demographics.columns = cure_precinct_demographics.columns.str.replace('% ','') 
cure_precinct_demographics_pre_covid.columns = cure_precinct_demographics_pre_covid.columns.str.replace('% ','') 

columns = {'Adults with high school diploma (includes equivalency)':'Adults with high school diploma',
           'Unemployed (civilians 16 and older)':'Unemployment Rate', 
           'With private health insurance (civilian, noninstitutionalized)':'Private health insurance',
           'With public coverage (civilian, noninstitutionalized)':'Public coverage',
           'With no health insurance coverage (civilian, noninstitutionalized)':'No health insurance'}
 
# changing some column names     
cure_precinct_demographics = cure_precinct_demographics.rename(columns=columns) 
cure_precinct_demographics_pre_covid = cure_precinct_demographics_pre_covid.rename(columns=columns)

# dropping column since finished using it to calulcate unemployment rate
cure_precinct_demographics = cure_precinct_demographics.drop(columns=['In the civilian labor force (16 and older)']) # dropping columns
cure_precinct_demographics_pre_covid = cure_precinct_demographics_pre_covid.drop(columns=['In the civilian labor force (16 and older)']) # dropping columns

# deleting from t test dictionaries too
del t_stat_dict['% In the civilian labor force (16 and older)']
del p_val_dict['% In the civilian labor force (16 and older)']
del t_stat_dict_pre_covid['% In the civilian labor force (16 and older)']
del p_val_dict_pre_covid['% In the civilian labor force (16 and older)']

# rounding
cure_precinct_demographics['Total population'] = round(cure_precinct_demographics['Total population']) 
cure_precinct_demographics_pre_covid['Total population'] = round(cure_precinct_demographics_pre_covid['Total population']) 

In [13]:
# pivotting tables

cure_balance_table = cure_precinct_demographics.swapaxes("index", "columns")
cure_balance_table_pre_covid = cure_precinct_demographics_pre_covid.swapaxes("index", "columns")

In [14]:
# adding test test columns 

cure_balance_table['t-stat'] = t_stat_dict.values()
cure_balance_table['p-value'] = p_val_dict.values()

cure_balance_table_pre_covid['t-stat'] = t_stat_dict_pre_covid.values()
cure_balance_table_pre_covid['p-value'] = p_val_dict_pre_covid.values()

In [15]:
# formatting numbers

for ind in cure_balance_table.index:
    
    if ind == 'Total population':
        
        cure_balance_table['No'][ind] = '{:,.0f}'.format(float(cure_balance_table['No'][ind]))
        cure_balance_table['Yes'][ind] = '{:,.0f}'.format(float(cure_balance_table['Yes'][ind]))
        
    elif ind == 'Median household income':
        
        cure_balance_table['No'][ind] = '$' + '{:,.0f}'.format(float(cure_balance_table['No'][ind][1:]))
        cure_balance_table['Yes'][ind] = '$' + '{:,.0f}'.format(float(cure_balance_table['Yes'][ind][1:]))

/var/folders/0x/wbh6lcrn3t7046vw2zxtbrhxrc0v0g/T/ipykernel_23316/1935647120.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cure_balance_table['No'][ind] = '{:,.0f}'.format(float(cure_balance_table['No'][ind]))
/var/folders/0x/wbh6lcrn3t7046vw2zxtbrhxrc0v0g/T/ipykernel_23316/1935647120.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cure_balance_table['Yes'][ind] = '{:,.0f}'.format(float(cure_balance_table['Yes'][ind]))
/var/folders/0x/wbh6lcrn3t7046vw2zxtbrhxrc0v0g/T/ipykernel_23316/1935647120.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in 

In [16]:
# formatting numbers

for ind in cure_balance_table_pre_covid.index:
    
    if ind == 'Total population':
        
        cure_balance_table_pre_covid['No'][ind] = '{:,.0f}'.format(float(cure_balance_table_pre_covid['No'][ind]))
        cure_balance_table_pre_covid['Yes'][ind] = '{:,.0f}'.format(float(cure_balance_table_pre_covid['Yes'][ind]))
        
    elif ind == 'Median household income':
        
        cure_balance_table_pre_covid['No'][ind] = '$' + '{:,.0f}'.format(float(cure_balance_table_pre_covid['No'][ind][1:]))
        cure_balance_table_pre_covid['Yes'][ind] = '$' + '{:,.0f}'.format(float(cure_balance_table_pre_covid['Yes'][ind][1:]))

/var/folders/0x/wbh6lcrn3t7046vw2zxtbrhxrc0v0g/T/ipykernel_23316/3588907389.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cure_balance_table_pre_covid['No'][ind] = '{:,.0f}'.format(float(cure_balance_table_pre_covid['No'][ind]))
/var/folders/0x/wbh6lcrn3t7046vw2zxtbrhxrc0v0g/T/ipykernel_23316/3588907389.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cure_balance_table_pre_covid['Yes'][ind] = '{:,.0f}'.format(float(cure_balance_table_pre_covid['Yes'][ind]))
/var/folders/0x/wbh6lcrn3t7046vw2zxtbrhxrc0v0g/T/ipykernel_23316/3588907389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [17]:
cure_balance_table

# cure_balance_table.to_csv('../data/output/cure_balance_table.csv')

Cure Precinct,No,Yes,t-stat,p-value
Total population,"112,550","121,191",0.73,0.47
Male,48.8%,46.4%,-5.93,0.00
Female,51.2%,53.6%,5.93,0.00
Hispanic or Latino,26.1%,32.1%,1.30,0.20
Asian alone,17.1%,6.3%,-4.06,0.00
American Indian or Alaska Native alone,0.5%,0.6%,0.94,0.35
Black or African American alone,11.1%,45.4%,9.66,0.00
Native Hawaiian or other Pacific Islander alone,0.1%,0.1%,0.01,0.99
White alone,51.3%,23.0%,-6.50,0.00
Some other race alone,12.4%,17.6%,1.80,0.08


In [18]:
cure_balance_table_pre_covid

# cure_balance_table_pre_covid.to_csv('../data/output/cure_balance_table_pre_covid.csv')

Cure Precinct Pre-Covid,No,Yes,t-stat,p-value
Total population,"115,235","117,038",0.14,0.89
Male,48.6%,46.2%,-5.08,0.00
Female,51.4%,53.8%,5.08,0.00
Hispanic or Latino,25.4%,36.0%,2.18,0.03
Asian alone,16.3%,4.9%,-3.92,0.00
American Indian or Alaska Native alone,0.5%,0.6%,0.84,0.41
Black or African American alone,16.2%,43.4%,5.66,0.00
Native Hawaiian or other Pacific Islander alone,0.1%,0.1%,0.56,0.58
White alone,47.2%,24.3%,-4.34,0.00
Some other race alone,12.5%,19.0%,2.10,0.04
